In [80]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error



In [81]:
#Gets the data from the scraper and deletes the index column
stats = pd.read_csv('merged.csv')
del stats['Unnamed: 0']
del stats['BPM']
stats = stats.fillna(0)
raptor = pd.read_csv('modern_RAPTOR_by_player.csv')

In [82]:
raptor_clean = raptor[['player_name','raptor_total', 'season', 'mp']]

columns_to_rename = {'player_name': 'Player','raptor_total': 'Raptor'}
raptor_clean = raptor_clean.rename(columns = columns_to_rename)
raptor_clean['Raptor'] = round(raptor_clean['Raptor'], 1)

In [83]:
#Missing data

#2000-2012 (Shuttlerun)
#2014, 16, 21, 22, 23 (Bench Press)

#Get total mp
#Remove duplicate names

shuttleRun = range(2000,2013)
benchPress = [2014,2016,2021,2022,2023]
stats.columns

Index(['Year', 'Player', 'POSITION', 'DRAFT_NUMBER', 'HEIGHT_WO_SHOES',
       'WEIGHT', 'WINGSPAN', 'STANDING_REACH', 'STANDING_REACH_FT_IN',
       'STANDING_VERTICAL_LEAP', 'MAX_VERTICAL_LEAP', 'LANE_AGILITY_TIME',
       'THREE_QUARTER_SPRINT', 'BENCH_PRESS', 'Age (Days)', 'DRAFT_NUMBER.1',
       'G', 'MP', 'FG%', '2P%', '3P%', 'FT%', 'PTS', 'TRB', 'AST', 'STL',
       'BLK', 'WS', 'Total Minutes', 'Seasons Played', 'Average Minutes'],
      dtype='object')

In [94]:
#Gets Predidctor variables
def get_predictors(year):
        columns_to_drop = ['Total Minutes', 
                         'Seasons Played',
                         'Player',
                         'POSITION',
                         'Year',
                         'STANDING_REACH_FT_IN',
                         'DRAFT_NUMBER.1',
                         'Average Minutes']

        predictors = (stats.drop(columns = columns_to_drop)).columns
        return predictors
        '''
        if year in shuttleRun:
            predictors.drop(columns = year)
        elif year in benchPress:
        '''    

        
    


In [95]:
df1 = stats[stats.isna().any(axis=1)]
df1

,Year,Player,POSITION,DRAFT_NUMBER,HEIGHT_WO_SHOES,WEIGHT,WINGSPAN,STANDING_REACH,STANDING_REACH_FT_IN,STANDING_VERTICAL_LEAP,...,FT%,PTS,TRB,AST,STL,BLK,WS,Total Minutes,Seasons Played,Average Minutes


In [105]:
def choose_year(year):
    
    #Training the model 
    train = stats[stats["Year"] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha = 0.01)
    reg.fit(train[predictors], train['Average Minutes'])

    #Getting predicitons in a dataframe
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns = ['Predictions'], index = test.index)
    combination = pd.concat([test[['Player', 'Average Minutes']], predictions], axis = 1)
    
    #Getting Mean Square Error
    mse = mean_squared_error(combination['Average Minutes'], combination['Predictions'])
    string = 'The mean square error for this draft is: ' + str(mse)
    
    #Displays rank which refers to each players draft position relative to one another in the given dataset as well as the predicted rank and difference
    #Not used cause clutter, can maybe delete?
    combination = combination.sort_values('Average Minutes', ascending = False)
    combination['RK'] = list(range(1, combination.shape[0]+1))
    combination = combination.sort_values('Predictions', ascending = False)
    combination['Predicted_RK'] = list(range(1, combination.shape[0]+1))
    combination['Difference'] = combination['Predicted_RK'] - combination['RK']

    #Gets a clutter free df 'clean' which neatly displays information
    important_columns = ['Player',  'Average Minutes',  'Predictions']
    clean = combination.loc[:,important_columns]
    clean['Predictions'] = round(clean['Predictions'], 0)
    clean['Average Minutes'] = round(clean['Average Minutes'], 0)
    clean['Predictions'] = clean['Predictions'].astype(int)
    clean['Average Minutes'] = clean['Average Minutes'].astype(int)
    clean['Difference'] = clean['Predictions'] - clean['Average Minutes'] 
    
    #Raptor, next step is use outer merge to get players with 0 min play

    return clean, string

    

In [109]:
year = input("Enter the draft class you would like to predict: ")


Enter the draft class you would like to predict: 2019


In [110]:
predictors = get_predictors(year)
predictors

Index(['DRAFT_NUMBER', 'HEIGHT_WO_SHOES', 'WEIGHT', 'WINGSPAN',
       'STANDING_REACH', 'STANDING_VERTICAL_LEAP', 'MAX_VERTICAL_LEAP',
       'LANE_AGILITY_TIME', 'THREE_QUARTER_SPRINT', 'BENCH_PRESS',
       'Age (Days)', 'G', 'MP', 'FG%', '2P%', '3P%', 'FT%', 'PTS', 'TRB',
       'AST', 'STL', 'BLK', 'WS'],
      dtype='object')

In [111]:
choose_year(int(year))

(                  Player  Average Minutes  Predictions  Difference
 415            Ty Jerome              638         1285         647
 428       Grant Williams             1354         1115        -239
 408       Brandon Clarke             1322         1107        -215
 429        Dylan Windler              489         1041         552
 417        Nassir Little              769          995         226
 413         Jaxson Hayes             1146          966        -180
 421        Jaylen Nowell              629          853         224
 409       Carsen Edwards              236          818         582
 416    Mfiondu Kabengele              172          802         630
 427       Tremont Waters              136          787         651
 407     Ignas Brazdeikis              278          787         509
 410       Bruno Fernando              364          753         389
 424          Isaiah Roby              795          643        -152
 411       Daniel Gafford              950      

AttributeError: 'NoneType' object has no attribute 'dtype'

In [11]:
test = 0
for year in range(2014,2021):
    print(year)
    print(choose_year(int(year))[1])
    test = test + choose_year(int(year))[1]
test

2014
-179174.38709677415
2015
-105098.39285714287
2016
-38748.25
2017
-65277.57575757573
2018
-96952.0
2019
-54204.181818181816
2020
-30987.999999999996


-570442.7875296746

In [63]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats
from nba_api.stats.endpoints import commonplayerinfo
def get_info(name):
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'] == name:
            player_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id'])
            df = player_info.get_data_frames()
            df[1]['SCHOOL'] = df[0]['SCHOOL']
    
    
    return(df[1])
    
    

In [64]:
a = get_info('Steven Adams')
a

,PLAYER_ID,PLAYER_NAME,TimeFrame,PTS,AST,REB,PIE,SCHOOL
0,203500,Steven Adams,2021-22,6.9,3.4,10.0,0.108,Pittsburgh


{'id': 76001,
 'full_name': 'Alaa Abdelnaby',
 'first_name': 'Alaa',
 'last_name': 'Abdelnaby',
 'is_active': False}